In [7]:
import numpy as np
import os

In [8]:
from evo.tools import file_interface
from evo.core import sync


In [9]:
# traj_ref = file_interface.read_tum_trajectory_file(DIR+"groundtruth.txt")
traj_ref = file_interface.read_tum_trajectory_file("../data/boxes_translation/groundtruth.txt")
# traj_est = file_interface.read_tum_trajectory_file("../data/boxes_translation/frame_trajectory_orbslam2_rgb.txt")
traj_est = file_interface.read_tum_trajectory_file("./boxes_translation/allison_predict_boxes_translation.txt")

traj_ref, traj_est = sync.associate_trajectories(traj_ref, traj_est, max_diff=1000)

In [10]:
print(traj_ref)
print(traj_est)

11956 poses, 0.004m path length, 0.055s duration
11956 poses, 43.558m path length, 0.056s duration


In [11]:
from evo.tools import log
log.configure_logging()
# interactive widgets configuration
import ipywidgets


from evo.tools import plot
from evo.tools.plot import PlotMode
from evo.core.metrics import PoseRelation, Unit
from evo.tools.settings import SETTINGS

# temporarily override some package settings
SETTINGS.plot_figsize = [6, 6]
SETTINGS.plot_split = True
SETTINGS.plot_usetex = False

# magic plot configuration
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook

import evo.main_ape as main_ape
import evo.common_ape_rpe as common

# interactive widgets configuration
import ipywidgets

check_opts_ape = {"align": False, "correct_scale": False, "show_plot": True}
check_boxes_ape=[ipywidgets.Checkbox(description=desc, value=val) for desc, val in check_opts_ape.items()]
check_opts_rpe = {"align": False, "correct_scale": False, "all_pairs": False, "show_plot": True}
check_boxes_rpe=[ipywidgets.Checkbox(description=desc, value=val) for desc, val in check_opts_rpe.items()]
delta_input = ipywidgets.FloatText(value=1.0, description='delta', disabled=False, color='black')
delta_unit_selector=ipywidgets.Dropdown(
    options={u.value: u for u in Unit if u is not Unit.seconds},
    value=Unit.frames, description='delta_unit'
)
plotmode_selector=ipywidgets.Dropdown(
    options={p.value: p for p in PlotMode},
    value=PlotMode.xy, description='plot_mode'
)
pose_relation_selector=ipywidgets.Dropdown(
    options={p.value: p for p in PoseRelation},
    value=PoseRelation.translation_part, description='pose_relation'
)

In [12]:

count = 0
results = []

def callback_ape(pose_relation, align, correct_scale, plot_mode, show_plot):
    global results, count
    est_name="APE Test #{}".format(count)
    
    result = main_ape.ape(traj_ref, traj_est, est_name=est_name,
                          pose_relation=pose_relation, align=align, correct_scale=correct_scale)
    count += 1
    results.append(result)
    
    if show_plot:
        fig = plt.figure()
        ax = plot.prepare_axis(fig, plot_mode)
        plot.traj(ax, plot_mode, traj_ref, style="--", alpha=0.5)
        plot.traj_colormap(
            ax, result.trajectories[est_name], result.np_arrays["error_array"], plot_mode,
            min_map=result.stats["min"], max_map=result.stats["max"])
    
_ = ipywidgets.interact_manual(callback_ape, pose_relation=pose_relation_selector, plot_mode=plotmode_selector,
                               **{c.description: c.value for c in check_boxes_ape})

interactive(children=(Dropdown(description='pose_relation', index=1, options={'full transformation': <PoseRela…

In [ ]:
import os
dd = "../data/event_fused_pos_neg/"
ll = os.listdir(dd+"images/")
ll.sort()
ts = [str(float(l[:-4])/10.0) for l in ll]    

In [ ]:
with open(dd+"rgb.txt", "w") as fp:
    for i in range(len(ll)):
        l = str(ts[i])+" images/"+ll[i]+"\n"
        fp.write(l)
with open(dd+"depth.txt", "w") as fp:
    for i in range(len(ll)):
        l = str(ts[i])+" images/"+ll[i]+"\n"
        fp.write(l)

In [2]:
with open("/Users/arpit/Spring20/SLAM_16833/project/data/boxes_translation/update_bag_translation_3.txt","r") as fp:
    lll = fp.readlines()
    for i,l in enumerate(lll):
        if not len(l[:-1].split(" ")) == 8:
            print(str(i+1)+",,"+l)
        try:
            aaa = [float(t) for t in l[:-1].split(" ")]
        except:
            print(i,l)
#     lll = [[float(t) for t in l[:-1].split(" ")] for l in lll]
    

3527  -0.104964 0.00871518 0.00917347 0.900695 0.0406732 -0.00410594 -0.432665

5757,,-0.0462941 0.32884 0.177633 0.857268 0.00818209 0.00885068 -0.516703

9190,,0.0768912 -0.818106 0.0805707 -0.182489 0.5395937, 0.412868, 0.122264

9189 0.0768912 -0.818106 0.0805707 -0.182489 0.5395937, 0.412868, 0.122264

12300,,-0.0859314 -0.100471 0.484775 0.794509 -0.0248362 0.0265521 -0.606776

21820,,

21819 

21821,,

21820 



In [ ]:
ref = lll[0][0]
for ll in lll:
    ll[0] = (ll[0] - ref)/1e9

In [ ]:
with open("./boxes_translation/boxes_translation_5000_pos2.txt","w") as fp:
    mmm = [" ".join([str(i) for i in ll])+"\n" for ll in lll]
    for mm in mmm:
        fp.write(mm)

In [ ]:
# ORBSLAM2 + RGB
       max	0.453927
      mean	0.045071
    median	0.039625
       min	0.021291
      rmse	0.051572
       sse	3.343230
       std	0.025066
#ORBSLAM2 + Events5000:
    not initialized
#ORBSLAM2 + Eventspos+neg:
    not initialized


In [ ]:
import os
import cv2 

indir = "../data/event_fused_pos_neg/images/"
outdir= "../data/event_fused_pos_neg/images_blur/"
# os.mkdir(outdir)
im_in = [indir+f for f in os.listdir(indir)]
im_out = [outdir+f for f in os.listdir(indir)]
for i,impath in enumerate(im_in):
    img = cv2.GaussianBlur(cv2.imread(impath), (5,5), 0)
    cv2.imwrite(im_out[i], img, )
#     break
